In [38]:
import yfinance as yf
import numpy as np
from scipy.optimize import minimize
import pandas as pd
from pypfopt import expected_returns, risk_models, BlackLittermanModel, EfficientFrontier
import yfinance as yf
import talib
import pandas as pd

# <u>Portfolio Optimisation</u>

https://theaiquant.medium.com/mastering-complete-portfolio-optimization-with-mean-variance-analysis-in-python-4d78c5e7a688

This notebook will look at creating an optimal portfolio of assets that maximises return for a given level of risk.

A optimisation model will be created using techniques including Machine Learning, Black-Litterman Model, and Monte Carlo Simulations alongside traditional Mean-Variance Optimization.

## A. Portfolio Optimisation using Traditional Mean-Variance Optimization (Markowitz Modern Portfolio Theory)

Traditional Methods of Portfolio optimisation

Harry Markowitz modern portfolio theory, which uses mean-variance optimiation to minimizes risk (variance) for a given return. Optimizes the portfolio's asset weights based on expected returns and the covariance matrix of returns.

To do this scipy's optimize function is used to maximise the sharpe-ratio (which is the return to risk ratio).

### Step 1: Data Collection

In [17]:
assets = ['BARC.L', 'BP', 'GOOGL', 'LLOY.L', 'META','SHEL', 'VOD', 'VUSA.AS']
data = yf.download(assets, start="2015-01-01", end="2024-01-01")['Adj Close']

[*********************100%%**********************]  8 of 8 completed


Using a time frame of around 10 years to reflect Long-term perspective of asset behaviours and modelling over this timeframe

In [18]:
# Handle missing values (forward-fill and backward-fill)
data = data.ffill().bfill()

# Calculating the log returns benefits include: time additive, allow for easier aggregation of returns across periods.
returns = np.log(data / data.shift(1))

# Compute mean returns and covariance matrix
mean_returns = returns.mean()
cov_matrix = returns.cov()

In [7]:
# Define portfolio return and volatility functions
def portfolio_performance(weights, mean_returns, cov_matrix):
    portfolio_return = np.sum(weights * mean_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return portfolio_return, portfolio_volatility

# Objective function (maximize Sharpe ratio)
def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0.03):
    p_ret, p_volatility = portfolio_performance(weights, mean_returns, cov_matrix)
    return -(p_ret - risk_free_rate) / p_volatility

# Constraints (weights sum to 1)
def check_sum(weights):
    return np.sum(weights) - 1

# Bounds for weights (each weight between 0 and 1)
bounds = tuple((0, 1) for asset in range(len(assets)))

# Initial guess (equal weight distribution)
initial_weights = len(assets) * [1. / len(assets)]

# Optimize
constraints = ({'type': 'eq', 'fun': check_sum})
result = minimize(negative_sharpe_ratio, initial_weights, args=(mean_returns, cov_matrix),
                  method='SLSQP', bounds=bounds, constraints=constraints)

optimal_weights = result.x

In [8]:
optimal_weights
# sum(optimal_weights)

array([1.00000000e+00, 9.34297084e-12, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 5.94164561e-12, 0.00000000e+00, 0.00000000e+00])

In [9]:
# Initial guess (equal weight distribution)
initial_weights = len(assets) * [1. / len(assets)]

In [10]:
initial_weights

[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]

In [13]:
# DataFrame to display tickers and their corresponding optimal weights
weights_df = pd.DataFrame({
    'Ticker': assets,
    'Weight': optimal_weights
})
weights_df = weights_df.sort_values(by='Weight', ascending=False)
print(weights_df)

    Ticker        Weight
0   BARC.L  1.000000e+00
1       BP  9.342971e-12
5     SHEL  5.941646e-12
2    GOOGL  0.000000e+00
3   LLOY.L  0.000000e+00
4     META  0.000000e+00
6      VOD  0.000000e+00
7  VUSA.AS  0.000000e+00


## B. Portfolio Optimisation using Black-Litterman Model

Black-Litterman Model

Black-Litterman model is used to optimise asset allocation within an investor’s risk tolerance and market views.

In [36]:
# 1. Calculate expected returns and covariance matrix
mu = expected_returns.mean_historical_return(data)
S = risk_models.sample_cov(data)

# 2. Define market equilibrium returns (pi)
# The market caps are required in the model as it helps calculate the market-implied equilibriums returns.
# The returns of which represent the prior belief. 

# Define your stock tickers
tickers = ['BARC.L', 'BP', 'GOOGL', 'LLOY.L', 'META','SHEL', 'VOD', 'VUSA.AS']

# Fetch market cap data
market_caps = {}
for ticker in tickers:
    stock = yf.Ticker(ticker)
    try:
        market_cap = stock.info["marketCap"]  # Market capitalization
        market_caps[ticker] = market_cap
    except KeyError:
        print(f"Market cap not available for {ticker}")

market_capss = []
# Print market caps
print("Market Capitalizations:")
for ticker, cap in market_caps.items():
    print(f"{ticker}: {cap}")
    market_capss.append(cap)

market_capss

Market cap not available for VUSA.AS
Market Capitalizations:
BARC.L: 38446784512
BP: 76128632832
GOOGL: 2080344375296
LLOY.L: 32299640832
META: 1436939714560
SHEL: 198653181952
VOD: 23172112384


[38446784512,
 76128632832,
 2080344375296,
 32299640832,
 1436939714560,
 198653181952,
 23172112384]

In [20]:
# Instead of using the default risk aversion below. We can estimate risk aversion using historical data as follows:

# Example tickers (representing the market)
market_ticker = "^GSPC"  # S&P 500 as a proxy for market portfolio
risk_free_ticker = "^TNX"  # 10-year Treasury Yield data (proxy for risk-free rate)

# Download historical data for market (e.g., S&P 500) and risk-free rate (e.g., 13-week Treasury bill)
market_data = yf.download(market_ticker, start="2010-01-01", end="2024-01-01")['Adj Close']
risk_free_data = yf.download(risk_free_ticker, start="2010-01-01", end="2024-01-01")['Adj Close'] / 100  # Convert to percentage

# Calculate daily returns for market and risk-free rate
market_returns = market_data.pct_change().dropna()
risk_free_rate = risk_free_data.pct_change().dropna()

# Calculate the expected market return (annualized)
expected_market_return = market_returns.mean() * 252  # Annualize by multiplying by 252 trading days
print(f"Expected Market Return: {expected_market_return}")

# Calculate the variance of the market portfolio (annualized)
market_variance = market_returns.var() * 252  # Annualize by multiplying by 252 trading days
print(f"Market Variance: {market_variance}")

# Risk-free rate (annualized)
annual_risk_free_rate = risk_free_rate.mean() * 252
print(f"Annual Risk-Free Rate: {annual_risk_free_rate}")

# Estimate risk aversion (using Sharpe ratio approach)
risk_aversion = (expected_market_return - annual_risk_free_rate) / market_variance
print(f"Estimated Risk Aversion: {risk_aversion:.2f}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Expected Market Return: 0.11829804390813935
Market Variance: 0.030681580979339276
Annual Risk-Free Rate: 0.10792757349171224
Estimated Risk Aversion: 0.34


In [ ]:
# We need to define investor views which are done using arrays P and Q
# To get investor views. We will use technical analysis 

In [39]:
# Fetch market data
# tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA"]
tickers = ['BARC.L', 'BP', 'GOOGL', 'LLOY.L', 'META','SHEL', 'VOD', 'VUSA.AS']
data = yf.download(tickers, start="2023-01-01", end="2024-11-27")["Adj Close"]
data = data.ffill().dropna()

# Calculate RSI and MACD for each ticker
rsi_values = {}
macd_values = {}

for ticker in tickers:
    prices = data[ticker]
    
    # Calculate RSI using TA-Lib
    rsi = talib.RSI(prices, timeperiod=14)
    rsi_values[ticker] = rsi.iloc[-1]  # Most recent RSI value
    
    # Calculate MACD using TA-Lib
    macd, macd_signal, macd_hist = talib.MACD(
        prices, 
        fastperiod=12, 
        slowperiod=26, 
        signalperiod=9
    )
    macd_values[ticker] = macd_hist.iloc[-1]  # Most recent MACD histogram value

print(rsi_values)
print(macd_values)

[*********************100%%**********************]  8 of 8 completed

{'BARC.L': 59.041509898650844, 'BP': 42.94519755590237, 'GOOGL': 47.15540912945262, 'LLOY.L': 37.58538617929556, 'META': 52.06967751444282, 'SHEL': 41.56169889792674, 'VOD': 47.93092059304563, 'VUSA.AS': 72.52363193797451}
{'BARC.L': -0.19092101374566273, 'BP': 0.11450634437948692, 'GOOGL': -1.4909294455657338, 'LLOY.L': 0.03896296516210129, 'META': -1.0401933129860177, 'SHEL': -0.0542031102300643, 'VOD': 0.03864660473723286, 'VUSA.AS': 0.14567996167239494}


In [44]:
views = []  # List to store views
for ticker in tickers:
    if rsi_values[ticker] < 70 and macd_values[ticker] > 0:
        views.append((ticker, "bullish"))
    elif rsi_values[ticker] > 30 and macd_values[ticker] < 0:
        views.append((ticker, "bearish"))

views

[('BARC.L', 'bearish'),
 ('BP', 'bullish'),
 ('GOOGL', 'bearish'),
 ('LLOY.L', 'bullish'),
 ('META', 'bearish'),
 ('SHEL', 'bearish'),
 ('VOD', 'bullish')]

In [45]:
import numpy as np

# Initialize P and Q
P = []
Q = []

# Define expected return assumptions
bullish_return = 0.08  # Example: 8% expected return for bullish views
bearish_return = -0.05  # Example: -5% expected return for bearish views

# Populate P and Q based on views
for view in views:
    row = [0] * len(tickers)  # Initialize row for P
    ticker, sentiment = view
    idx = tickers.index(ticker)  # Get index of the ticker in tickers
    
    if sentiment == "bullish":
        row[idx] = 1  # +1 for bullish view
        Q.append(bullish_return)
    elif sentiment == "bearish":
        row[idx] = -1  # -1 for bearish view
        Q.append(bearish_return)
    
    P.append(row)

P = np.array(P)
Q = np.array(Q)

print("P Matrix:")
print(P)
print("\nQ Vector:")
print(Q)


P Matrix:
[[-1  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0  0]
 [ 0  0  0  1  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0 -1  0  0]
 [ 0  0  0  0  0  0  1  0]]

Q Vector:
[-0.05  0.08 -0.05  0.08 -0.05 -0.05  0.08]


In [46]:
# 4. Initialize the Black-Litterman model
bl = BlackLittermanModel(S, Q=Q, P=P, market_caps=market_caps, risk_aversion=0.34)

# 5. Get adjusted returns and covariance matrix
bl_return = bl.bl_returns()
bl_cov = bl.bl_cov()

# 6. Optimize portfolio using Black-Litterman outputs
ef = EfficientFrontier(bl_return, bl_cov)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
ef.portfolio_performance(verbose=True)

print("Optimized Weights:", cleaned_weights)

Expected annual return: 5.2%
Annual volatility: 22.5%
Sharpe Ratio: 0.14
Optimized Weights: OrderedDict({'BARC.L': 0.0, 'BP': 0.26124, 'GOOGL': 0.01512, 'LLOY.L': 0.26091, 'META': 0.07004, 'SHEL': 0.0, 'VOD': 0.3927, 'VUSA.AS': 0.0})
